# **ETL del archivo Steam_Games_Analisis**

## **1.Cargando Las Librerías Necesarias**

In [1]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import json as json  #Cargaremos json para manejar los datos de tipo json",
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import ast #Cargaremos ast para poder extraer datos de json",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
import re
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url01=r'/content/drive/MyDrive/2023/Henry/MLOps Henry/Data/steam_games.json.gz'

## **2.Fase ETL**

### 2.1.Carga de Datos

In [3]:
#al revisar los archivos nos damos cuenta que el archivo 1 es diferente a los otros 2 por tanto usaremos otro proceso
df_steam_games=pd.read_json(url01,lines=True)
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


#### 2.2.1.Funciones Para El Analisis Descriptivo De Los Datos

### 3.1.Definiremos funciones para el analisis descriptivo de los datos

In [4]:
def check_df(dataset, head = 5):
    """
    Funcion creada para tener una vista general de la base de datos
    """
    print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
    print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

    print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
    print(dataset.info(), end = '\n'*2)# Informacion general

    print('*'*30 + 'Mostrar las primeras 5 filas' + '*'*30, end = '\n'*2)
    print(dataset.head(), end = '\n'*2)# Mostrar las primeras 5 filas

    print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
    print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [5]:
check_df(df_steam_games)

******************************Forma de la base de datos******************************

(120445, 13)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usag

### 2.3.Transformacion de Datos

#### 2.3.1.Funciones para la transformacion de datos

In [6]:
def setting_price(text):

  """
  Funcion creada para extraer el precio de los juegos donde text es un dato de tipo string
  de la columna 'price'
  donde se espera que las etiquetas :
  values_to_replace=[ 'Free to play','Free to Use','Play for Free!','Free','None','Third-party',
                  'Install Now','Play WARMACHINE: Tactics Demo','Play Now','Free HITMAN™ Holiday Pack',
                  'Free Demo','Free Movie','Free to Try','Play the Demo','Install Theme','Free Mode']
  'Starting at $499.00','Starting at $449.00'
  nos retorne 0 , 499 y 449 respectivamente
  en otro caso nos retorne un numero flotante con 2 decimales

  """

  text=str(text)
  text=text.lower()

  values_to_replace=['free to play', 'free to use','play for free!','free mod','free','none',
                    'third-party','install now','play warmachine: tactics demo',
                    'play now','free hitman™ holiday pack','free demo','free movie',
                    'free to try','play the demo','install theme','free mode']

  if text in values_to_replace:
    output  = 0

  elif text=='starting at $499.00':
    output  = 499

  elif text=='starting at $449.00':
    output  = 449

  else:
    output  = round(float(text),2)

  return output



def transform_to_zero(x):

  """
  Funcion creada para transformar los datos con caracteres especiales en 0
  donde x es un dato donde si es 0 nos retorna 0 ,si es un carater es tipo string
  y no tiene numeros en su contenido nos retorna 0   caso contrario nos retornara
  el mismo dato de entrada
  """

  if x == 0:
    y = 0
    return y

  if not re.search(r'\d', x):
    y = 0
    return y

  else:
    y=x

  return y



def extract_year(date_str):

  """
  Funcion creada para extraer el año de las fechas en las que puede hacerlo
  donde date_str es un dato de tipo string
  donde se espera que el retorno sea el año de la fecha en la que se encuentra
  en formato string

  """

  date_str = str(date_str)
  year_match = re.search(r'\d{4}', date_str)
  if year_match:
      return str(year_match.group())
  else:
      return None

#### 2.3.2.Transformacion

In [7]:
#Procederemos a dropear las columnas que posean mas de 3 columnas nulas

df_steam_games.dropna(thresh=3, inplace=True)

df_steam_games['price']=df_steam_games['price'].apply(setting_price)

df_steam_games.drop(['title','url','reviews_url','developer','specs','tags'], axis=1,inplace=True)

df_steam_games['early_access'] = df_steam_games['early_access'].astype(int)

df_steam_games['id'] = pd.to_numeric(df_steam_games['id'], errors='coerce')

df_steam_games['id']=df_steam_games['id'].astype('Int64')

df_steam_games['id']=df_steam_games['id'].astype(str)

df_steam_games['genres'].fillna('[]', inplace=True)

df_steam_games['release_date'].fillna(0, inplace=True)



In [8]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32135 entries, 88310 to 120444
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        32135 non-null  object 
 2   app_name      32133 non-null  object 
 3   release_date  32135 non-null  object 
 4   price         32135 non-null  float64
 5   early_access  32135 non-null  int64  
 6   id            32135 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.0+ MB


In [9]:
df_steam_games.head()

,publisher,genres,app_name,release_date,price,early_access,id
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,0,761140
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,0,643980
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,0,670290
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,0,767400
88314,None,[],Log Challenge,0,2.99,0,773570


In [10]:
wrong_dates = df_steam_games["release_date"][pd.to_datetime(df_steam_games["release_date"], errors='coerce').isna()]
wrong_dates

<ipython-input-10-e44fd55a4c40>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  wrong_dates = df_steam_games["release_date"][pd.to_datetime(df_steam_games["release_date"], errors='coerce').isna()]


88320          Soon..
88354       Beta测试已开启
95047     Coming Soon
95378         Q2 2017
95444     Coming Soon
             ...     
120368    Winter 2018
120369     2018 early
120391        Q1 2018
120393       Q1, 2018
120397    Coming Soon
Name: release_date, Length: 174, dtype: object

Se observa que existe gran cantidad de fechas invalidas para lo cual procederemos a tratar de transformarlas lo mas cercanos a fechas
para lo cual es opto por usar el año como referencia

In [11]:
df_steam_games['release_date'] = df_steam_games['release_date'].apply(transform_to_zero)

In [12]:
 # Mostramos nuestras fechas invalidas
wrong_dates = df_steam_games["release_date"][pd.to_datetime(df_steam_games["release_date"], errors='coerce').isna()]
wrong_dates.unique()

<ipython-input-12-f929f4f303dc>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  wrong_dates = df_steam_games["release_date"][pd.to_datetime(df_steam_games["release_date"], errors='coerce').isna()]


array(['Q2 2017', 'Q2 2018', 'Winter 2017', '14 July', 'Summer 2017',
       'Spring 2018', 'Winter 2018', 'Fall 2017', 'Q1 2018',
       'Январь 2018', 'First quarter of 2018', 'H2 2018', '2018年初頭発売予定',
       '2018 [Now get free Pre Alpha]', '1st Quarter 2018', '预热群52756441',
       '2018年1月', 'Fall 2018',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       'Spring 2017', 'Coming 2017', 'Q4 2017', 'Early 2018',
       'Q2 2018 (Tentative)', '2017 Q4', 'Early Spring 2018', 'TBA 2017',
       'Coming Q1 2018', 'Coming Q3 2017', 'Early 2017', 'End 2017',
       '2017 Q1', 'Q1 2017', "When it's done (2017)", 'Q1 (ish), 2017',
       'Late 2016 - Early 2017', 'Coming Fall 2017', 'Coming late 2017',
       'Hitting Early Access in 2017', '2018 early', 'Q1, 2018'],
      dtype=object)

In [13]:
# Aplicar la función a la columna release_date y creamos una nueva columna año asi tendremos una informacion uniforme

df_steam_games['year'] = df_steam_games['release_date'].apply(extract_year)

Explotamos nuestros generos

In [14]:
df_steam_games = df_steam_games.explode('genres')

df_steam_games.drop('release_date', axis=1, inplace=True) # Eliminamos release_date ya que tenemos el año en una nueva columna

In [15]:
df_steam_games.year.unique() # Visualizamos nuestros años unicos (1970-2021)

array(['2018', '2017', None, '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '5275', '2019', '1975', '1970', '1980'], dtype=object)

In [16]:
df_steam_games.loc[df_steam_games.year == '5275', 'year'] = None

In [17]:
df_steam_games['year']=df_steam_games['year'].astype('Int64')

In [18]:
df_steam_games.to_parquet('/content/drive/MyDrive/2023/Henry/MLOps Henry/Data_Clean/clean_steam_games.parquet.gzip', compression='gzip')